In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

(95851, 6)


In [4]:
list_sentences_train[:5]

array([ 'nonsense kiss off geek what i said is true i will have your account terminated ',
       ' please do not vandalize pages as you did with this edit to w s merwin if you continue to do so you will be blocked from editing ',
       ' points of interest i removed the points of interest section you added because it seemed kind of spammy i know you probably did not mean to disobey the rules but generally a point of interest tends to be rather touristy and quite irrelevant to an area culture that just my opinion though if you want to reply just put your reply here and add talkback jamiegraham08 on my talkpage ',
       'asking some his nationality is a racial offence wow was not aware of it blocking me has shown your support towards your community thanku for that',
       'the reader here is not going by my say so for ethereal vocal style and dark lyrical content the cited sources in the external links are saying those things if you feel the sources are unreliable or i did not repres

In [5]:
print('test len',len(test))

test len 226998


In [6]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

(95851, 150) (226998, 150)


In [7]:
# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

keynote 30116
catanduanes 108825
cccc 26059
mathewfenton 88408
domed 89978
gashapon 113618


In [8]:
# read word2vec
# 
word_vec_dict = {}
with open('../glove.840B.300d.txt') as f:
    for line in f:
        v_list = line.strip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

2196007
Preparing embedding matrix
Null word embeddings: 3819


In [33]:
from sklearn.metrics import log_loss,accuracy_score
from keras.layers import concatenate
from keras import regularizers

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_cnn_model(comp=True):
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = Dropout(0.2)(x)
    
    x1 = Conv1D(256,
             3,
             padding='valid',
             activation='relu',
             strides=1)(x)
    x1 = GlobalMaxPool1D()(x1)
    
    x2= Conv1D(256,
             5,
             padding='valid',
             activation='relu',
             strides=1)(x)
    x2 = GlobalMaxPool1D()(x2)
    
    x3 = Conv1D(256,
             7,
             padding='valid',
             activation='relu',
             strides=1)(x)
    x3 = GlobalMaxPool1D()(x3)
    
    x = concatenate([x1,x2,x3])
    
    x = Dropout(0.5)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    if comp:
        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

    return model
print('def model done')


tmp_m = get_cnn_model()
tmp_m.summary()

def model done
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_26 (Embedding)        (None, 150, 300)     12000000    input_27[0][0]                   
__________________________________________________________________________________________________
dropout_65 (Dropout)            (None, 150, 300)     0           embedding_26[0][0]               
__________________________________________________________________________________________________
conv1d_76 (Conv1D)              (None, 148, 256)     230656      dropout_65[0][0]                 
______________________________________________________________________________________________

In [34]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((95851,6)),np.zeros((226998,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_cnn_model()
        batch_size = 64
        epochs = 10
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


train_pred,test_pred = kf_train()
print(train_pred.shape,test_pred.shape)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/cnn_glove_2_csv.gz", index=False, compression='gzip')
import pickle
with open('../features/cnn_glove_2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print('done')

Train on 63900 samples, validate on 31951 samples
Epoch 1/10
63900/63900 [==============================] - 79s 1ms/step - loss: 0.0741 - acc: 0.9751 - val_loss: 0.0585 - val_acc: 0.9799
Epoch 2/10
63900/63900 [==============================] - 77s 1ms/step - loss: 0.0553 - acc: 0.9798 - val_loss: 0.0474 - val_acc: 0.9820
Epoch 3/10
63900/63900 [==============================] - 77s 1ms/step - loss: 0.0510 - acc: 0.9807 - val_loss: 0.0476 - val_acc: 0.9820
Epoch 4/10
63900/63900 [==============================] - 77s 1ms/step - loss: 0.0486 - acc: 0.9814 - val_loss: 0.0472 - val_acc: 0.9816
Epoch 5/10
63900/63900 [==============================] - 78s 1ms/step - loss: 0.0463 - acc: 0.9818 - val_loss: 0.0457 - val_acc: 0.9824
Epoch 6/10
63900/63900 [==============================] - 78s 1ms/step - loss: 0.0440 - acc: 0.9826 - val_loss: 0.0474 - val_acc: 0.9823
Epoch 7/10
63900/63900 [==============================] - 78s 1ms/step - loss: 0.0424 - acc: 0.9831 - val_loss: 0.0462 - val_acc

In [35]:
train_pred[:10].round(3)

array([[ 0.536,  0.   ,  0.046,  0.   ,  0.165,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.018,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.003,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.043,  0.   ,  0.001,  0.   ,  0.001,  0.   ],
       [ 0.018,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [36]:
y[:10]

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])